# Enhancing performance using compiled code

We first look at native code compilation. Here we show 3 common methods for doing this using 

- `numba` JIT compilation
- `cython` AOT compilation, 
- Direct wrapping of C++ code using `cffi`. 

In general, `numba` is the simplest to use, while you have the most flexibility with `cffi`. Which approach gives the best performance generally requires some experimentation.

## Using `numba` Just-in-time compilation (JIT)
For programmer productivity, it often makes sense to code the majority of your application in a high-level language such as Python and only optimize code bottlenecks identified by profiling. One way to speed up these bottlenecks is to compile the code to machine executables, often via an intermediate C or C-like stage. There are two common approaches to compiling Python code - using a Just-In-Time (JIT) compiler and using `Cython` for Ahead of Time (AOT) compilation.

LLVM (Low Level Virtual Machine) library already implements a compiler backend. It is used to construct, optimize and produce intermediate and/or binary machine code.
- A compiler framework, where you provide the "front end" (parser and lexer) and the "back end" (code that converts LLVM's representation to actual machine code).
- Multi platform.
- LLVM optimizations (inlining, loop unrolling, SIMD vectorization etc).

<center><img src="https://i.stack.imgur.com/9xGDe.png"></center>

<div align="center"> source: https://stackoverflow.com/questions/2354725/what-exactly-is-llvm </div>

### `Numba`
- From the types of the function arguments, `numba` can often generate a specialized, fast, machine code implementation at runtime.
- Designed to work best with numerical code and `numpy` arrays.
- Does not require any C/C++ compiler.
- Does not replace the standard Python interpreter (all Python libraries are still available).

<center><img src="https://nbviewer.jupyter.org/github/akittas/presentations/blob/master/pythess/numba/figures/how_numba_works.png"></center>

<div align="center"> source: https://nbviewer.jupyter.org/github/akittas/presentations/blob/master/pythess/numba/numba.ipynb?utm_source=newsletter_mailer&utm_medium=email&utm_campaign=weekly </div>

- Object mode - Compiled code operates on Python objects. Supports nearly all of Python, but may not speed up code by a large factor. 
- nopython mode - Compiled code operates on native machine data. Supports a subset of Python, but runs close to C/C++/FORTRAN speed.

In [ ]:
from numba import jit, njit
import numpy as np

x = np.arange(256).reshape(16, 16)

def go(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   
        trace += np.tanh(a[i, i]) 
    return a + trace             

@jit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

@jit(nopython=True)# Set "nopython" mode for best performance, equivalent to @njit
def go_fast2(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

In [ ]:
%timeit go(x)
%timeit go_fast(x)
%timeit go_fast2(x)

### MoteCarlo PI

<center><img src="https://upload.wikimedia.org/wikipedia/commons/2/20/MonteCarloIntegrationCircle.svg"></center>

<div align="center"> source: https://upload.wikimedia.org/wikipedia/commons/2/20/MonteCarloIntegrationCircle.svg </div>


If we sample uniformly N points in $[-1,1] \times [-1,1]$ and find that M points are inside circle. Then, we get $\frac{M}{N} \sim \frac{\pi}{4}$ which can help us to estimate $\pi$ 

In [ ]:
def monte_carlo_pi(n):
    x = np.random.uniform(-1, 1, (n,2))
    return 4*np.sum((x**2).sum(1) < 1)/n

In [ ]:
n = int(1e7)
monte_carlo_pi(n)

In [ ]:
%timeit monte_carlo_pi(n)

In [ ]:
@jit(nopython=True)
def monte_carlo_pi_numba(n):
    x = np.random.uniform(-1, 1, (n,2))
    return 4*np.sum((x**2).sum(1) < 1)/n

In [ ]:
%timeit monte_carlo_pi_numba(n)

While NumPy has developed a strong idiom around the use of vector operations, Numba is perfectly happy with loops too. For users familiar with C or Fortran, writing Python in this style will work fine in Numba (after all, LLVM gets a lot of use in compiling C lineage languages). 

In [ ]:
@jit(nopython=True, cache=True)
def monte_carlo_pi_numba_unrolled(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.uniform(-1,1)
        y = np.random.uniform(-1,1)
        if (x*x + y*y) < 1:
            acc += 1
    return 4 * acc / nsamples

In [ ]:
%timeit monte_carlo_pi_numba_unrolled(n)

<div class="alert alert-success">
    
- Check "cache" flag in the `Numba` decorector: https://numba.readthedocs.io/en/stable/user/jit.html
    
</div>

### Vectorization
Numba’s vectorize allows Python functions taking scalar input arguments to be used as NumPy `ufuncs`. Creating a traditional NumPy ufunc is not the most straightforward process and involves writing some C code. Numba makes this easy. Using the `vectorize()` decorator, Numba can compile a pure Python function into a `ufunc` that operates over NumPy arrays as fast as traditional ufuncs written in C.

**Using `vectorize()`, you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs.**

In [ ]:
def Theta_for(x):
    a = []
    for i in x:
        if i >= 0:
            a.append(1)
        else:
            a.append(0)
    return a

In [ ]:
from numba import vectorize
@vectorize
def Theta_numba(x):
    """
    Scalar implemenation of the Heaviside step function.
    """
    if x >= 0:
        return 1
    else:
        return 0

In [ ]:
def Theta(x):
    """
    Scalar implemenation of the Heaviside step function.
    """
    if x >= 0:
        return 1
    else:
        return 0
    
Theta_vec = np.vectorize(Theta)

In [ ]:
%timeit Theta_for(np.random.randn(500))
%timeit Theta_numba(np.random.randn(500))
%timeit Theta_vec(np.random.randn(500))

### FastMath
In certain classes of applications strict IEEE 754 compliance is less important. As a result, it is possible to relax some numerical rigor with the view of gaining additional performance.
Use with care. Some functions may be less accurate.

In [ ]:
@njit
def do_sum(A):
    acc = 0.
    # without fastmath, this loop must accumulate in strict order
    for x in A:
        acc += np.sqrt(x)
    return acc

@njit(fastmath=True)
def do_sum_fast(A):
    acc = 0.
    # with fastmath, the reduction can be vectorized as floating point
    # reassociation is permitted. (Out-of-order)
    for x in A:
        acc += np.sqrt(x)
    return acc

A = np.arange(1000)

In [ ]:
%timeit do_sum(A)
%timeit do_sum_fast(A)

In [ ]:
np.allclose(do_sum(A), do_sum_fast(A))

## Using `cython`
`Cython` is an “optimizing static compiler ” that combines Python with C to generate optimized code. Since `Cython` is a superset of Python, all valid Python programs are also valid `Cython` programs. However, by providing hints and static typing, we can get much faster programs. In this sense one could also call it a Python with types.

Note that while `numba` often provides similar speedups with less work, an advantage of `Cython` is that it works for general python language while `numba` is target for numpy.

In addition to the basic use case of wrapping native code, `Cython` supports an additional use-case, namely interactive optimization. Basically, one starts out with a pure-Python script and incrementally adds `Cython` types to the bottleneck code to optimize only those code paths that really matter.
The code can be autogenerated and the wrapping code can (almost) be written in Python.

- How to build Cython modules

Using Cython consists of these steps:

* Write a `.pyx` source file
* Run the Cython compiler to generate a C file
* Run a C compiler to generate a compiled library
* Run the Python interpreter and ask it to import the module
* In the `Jupyter notebook`, we can use the `%%cython` cell magic to automate these steps.

In [ ]:
%load_ext cython

In [ ]:
def matrix_multiply(u, v, res):
    m, n = u.shape
    n, p = v.shape
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

In [ ]:
u = np.random.random((10,20))
v = np.random.random((20,5))

In [ ]:
res = np.zeros((u.shape[0], v.shape[1]))
matrix_multiply(u, v, res)

In [ ]:
res = np.zeros((u.shape[0], v.shape[1]))
%timeit -r3 -n3 matrix_multiply(u, v, res)

In [ ]:
%%cython -a

import numpy as np

def matrix_multiply1(u, v, res):
    m, n = u.shape
    n, p = v.shape
    for i in range(m):
        for j in range(p):
            res[i,j] = 0
            for k in range(n):
                res[i,j] += u[i,k] * v[k,j]
    return res

The `nogil` keyword tells Cython that a particular function or code section should be executed without the GIL. When the GIL is released, it is not possible to make any Python API calls, meaning that only C variables and C functions (declared with `cdef`) can be used.

In [ ]:
%%cython -a

import cython

@cython.boundscheck(False)
@cython.wraparound(False) #non-negative
def matrix_multiply1(double[:,:] u, double[:, :] v, double[:, :] res):
    cdef int i, j, k
    cdef int m, n, p

    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]

    with cython.nogil:
        for i in range(m):
            for j in range(p):
                res[i,j] = 0
                for k in range(n):
                    res[i,j] += u[i,k] * v[k,j]

In [ ]:
res = np.zeros((u.shape[0], v.shape[1]))
%timeit -r3 -n3 matrix_multiply1(u, v, res)

### Fibonacci series
The Fibonacci Sequence is the series of numbers:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

The following number is obtained by summing up the two numbers before it.

In [ ]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

@njit
def fib_numba(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

In [ ]:
%%cython

def cfib(int n):
    cdef int i, a, b
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

In [ ]:
%timeit fib(1000)
%timeit fib_numba(1000)
%timeit cfib(1000)

## Numba+ CFFI


One of the limitations of the above approach is when need some specialized function that is available in `Numpy` or `Scipy`, but that function has not been re-implemented in the `Numba` core library so it can be called in the so-called **nopython** mode. Basically this means that if we want to call one of these functions, we have to go through Numba's **object mode**, which typically cannot generate nearly as efficient code.


We have other options: 

- The first is to re-implement a function using `Numba`. 

- A second technique that involves using `CFFI` to call external C code directly within Numba code. 


We can find a library [RMath](https://github.com/JuliaLang/Rmath-julia) that implements the same funtions of `scipy.stats` in C.

Other techniques to bind C code is introduced [here](http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html)

### Compile

The following script from [here](http://nbviewer.jupyter.org/github/synapticarbors/rmath-cffi-example/blob/master/rmath-cffi-example.ipynb)

In [ ]:
%%writefile build_rmath.py

import glob
import os
import platform

from cffi import FFI


include_dirs = [os.path.join('Rmath-julia', 'src'),
                os.path.join('Rmath-julia', 'include')]

rmath_src = glob.glob(os.path.join('Rmath-julia', 'src', '*.c'))

rmath_src = [f for f in rmath_src if ('librandom.c' not in f) and ('randmtzig.c' not in f)]

extra_compile_args = ['-DMATHLIB_STANDALONE']
extra_compile_args.append('-std=c99')


ffi = FFI()
ffi.set_source('_rmath_ffi', '#include <Rmath.h>',
        include_dirs=include_dirs,
        sources=rmath_src,
        libraries=[],
        extra_compile_args=extra_compile_args)

# We can add more available functions in Rmath here
ffi.cdef('''\
// Gamma Distribution
double dgamma(double, double, double, int);
double pgamma(double, double, double, int, int);
''')

if __name__ == '__main__':
    ffi.compile(verbose=False)

In [ ]:
!git clone https://github.com/phonchi/Rmath-julia.git

In [ ]:
!python build_rmath.py

Now that we have built our module wrapping `Rmath` using `cffi`, we can write Numba versions of scipy stats functions that we can call without additional overhead

In [ ]:
# Import our Rmath module
import _rmath_ffi

dgamma = _rmath_ffi.lib.dgamma
pgamma = _rmath_ffi.lib.pgamma

In order for us to use these methods from within a function that we can use JIT with `Numba`, we need to import `cffi_support` and register the module:

In [ ]:
from numba.core.typing import cffi_utils as cffi_support
cffi_support.register_module(_rmath_ffi)

In [ ]:
def cffi_gamma(x,alpha,beta):
    y = np.empty_like(x)
    for k in range(x.shape[0]):
        y[k] = dgamma(x[k], alpha, 1./beta, 0)
        
    return y

In [ ]:
@jit(nopython=True)
def nb_gamma_jit(x,alpha,beta):
    y = np.empty_like(x)
    for k in range(x.shape[0]):
        y[k] = dgamma(x[k], alpha, 1./beta, 0)
        
    return y

@vectorize(nopython=True)
def nb_gamma_vec(x,alpha,beta):
    return dgamma(x, alpha, 1./beta, 0) ##double x, double shape, double scale, int give_log

In [ ]:
import scipy.stats as st

x = np.random.normal(size=(100,))

def gamma_pdf(x,alpha,beta):
    gamma = st.gamma(alpha, loc=0, scale=1./beta)
    return gamma.pdf(x)

y1 = gamma_pdf(x,1.0,1.0)
y2 = cffi_gamma(x, 1.0, 1.0)
y3 = nb_gamma_vec(x, 1.0, 1.0)
y4 = nb_gamma_jit(x, 1.0, 1.0)
# Check that they all give the same results
print (np.allclose(y1, y2))
print (np.allclose(y1, y3))
print (np.allclose(y1, y4))

In [ ]:
%timeit gamma_pdf(x,1.0,1.0)
%timeit cffi_gamma(x,1.0,1.0)
%timeit nb_gamma_jit(x,1.0,1.0)
%timeit nb_gamma_vec(x, 1.0, 1.0)

## Laboratories
Given that we have two, 2D arrays. `x` has a shape of $(M, P)$ and `y` has a shape of $(N, P)$. We want to compute the Euclidean distance (a.k.a. the $L_2$-distance) between *each pair* of rows between the two arrays. That is, if a given row of `x` is represented by $P$ numbers $(x_0, x_1, \ldots, x_{P-1})$, and similarly, a row `y` is represented by $(y_0, y_1, \ldots, y_{P-1})$, and we want to compute the Euclidean distance between the two rows:

\begin{equation}
\sqrt{(x_{0} - y_{0})^2 + (x_{1} - y_{1})^2 + \ldots + (x_{P-1} - y_{P-1})^2} = \sqrt{\sum_{i=0}^{P-1}{(x_{i} - y_{i})^2}}
\end{equation}


In [ ]:
# Baseline
def cdist(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            res[i, j] = np.sqrt(np.sum((ys[j] - xs[i])**2))
    return res

In [ ]:
xs = np.arange(6).reshape(3,2).astype('float')
ys = np.arange(4).reshape(2,2).astype('float')
zs = cdist(xs, ys)

In [ ]:
zs

In [ ]:
%timeit -r 3 -n 10 cdist(xs, ys)

In [ ]:
%%cython -a

import cython
import numpy as np
from libc.math cimport sqrt, pow

@cython.boundscheck(False)
@cython.wraparound(False)
def cdist_cython(double[:, :] xs, double[:, :] ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    cdef int m, n, p
    
    m = xs.shape[0]
    n = ys.shape[0]
    p = xs.shape[1]
    
    cdef double[:, :] res = np.empty((m, n))
    
    cdef int i, j
    
    cdef double s
    for i in range(m):
        for j in range(n):
            s = 0.0
            for k in range(p):
                s += pow(ys[j,k] - xs[i,k], 2)                
            res[i, j] = sqrt(s)
    return res

In [ ]:
np.allclose(cdist(xs, ys), cdist_cython(xs, ys))

In [ ]:
%timeit  cdist_cython(xs, ys)

In [ ]:
m = 1000
n = 1000
p = 100

X = np.random.random((m, p))
Y = np.random.random((n, p))

In [ ]:
%timeit cdist(X, Y)
%timeit cdist_cython(X, Y)

## Exercise 2: 
- Calculate the pairwise euclidean distance between two matrices X and Y using `Numba` and report the speedup over baseline
- Hint: to get better performance, you can try writing the code with loops 

In [ ]:
## Solution here
# Write the Numba acceleration code for cdist(xs, ys)
@njit
def cdist_numba(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            res[i, j] = np.sqrt(np.sum((ys[j] - xs[i])**2))
    return res

In [ ]:
np.allclose(cdist(X, Y), cdist_numba(X, Y))

In [ ]:
## Timing profiling for the above Numba code and report the speedup
%timeit cdist_numba(X, Y)

The speedup is 7.06/0.314~ 22.5 times faster

In [ ]:
### Loop unrolling

In [ ]:
@njit
def cdist_numba1(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            s = 0
            for k in range(p):
                s += (ys[j,k] - xs[i,k])**2
            res[i, j] = np.sqrt(s)
    return res

In [ ]:
np.allclose(cdist(X, Y), cdist_numba1(X, Y))

In [ ]:
%timeit cdist_numba1(X, Y)

## References
- https://numba.readthedocs.io/en/stable/user/5minguide.html# - A great introduction for `Numba` from official site
- https://people.duke.edu/~ccc14/sta-663-2018/notebooks/S13A_Numba.html# - A series of great introduction for HPC
- https://nbviewer.jupyter.org/github/akittas/presentations/blob/master/pythess/numba/numba.ipynb?utm_source=newsletter_mailer&utm_medium=email&utm_campaign=weekly - An indepth discusion about `Numba`
- https://ipython-books.github.io/55-accelerating-python-code-with-cython/ - A gentle introduction to `Cython`
- https://people.duke.edu/~ccc14/sta-663-2018/notebooks/S13B_Cython.html - A series of great introduction for HPC
- https://people.duke.edu/~ccc14/bios-823-2020/notebooks/E05_Code_Optimization.html?highlight=performance - A series of great introduction for HPC